In [233]:
#Import libraries
from pyhive import presto
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import h3
from datetime import datetime
from functools import reduce
import json
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 500)

In [234]:
#Define presto credentials
presto_host = 'presto-gateway.serving.data.plectrum.dev'
presto_port = '80'
username = 'aditya.bhattar@rapido.bike'

#Create connection to presto host
connection = presto.connect(presto_host,presto_port,username = username)

In [235]:
df = pd.read_csv('mc_hexes.csv')
hexes = df['hex_id'].unique().tolist()
print(hexes)

['883da11a83fffff', '883da11ab1fffff', '883da118cbfffff', '883da118c9fffff', '883da11ab5fffff', '883da11aabfffff', '883da11abbfffff', '883da11abdfffff', '883da1188bfffff', '883da11aa1fffff', '883da11aa3fffff', '883da11aa7fffff', '883da11aa5fffff', '883da118cdfffff', '883da118c1fffff', '883da118c3fffff', '883da11ab7fffff', '883da1181bfffff', '883da118d7fffff', '883da118c7fffff', '883da11a87fffff', '883da118ddfffff', '883da118d3fffff', '883da118c5fffff', '883da11ab9fffff', '883da11889fffff', '883da11a97fffff', '883da118d1fffff', '883da11a95fffff', '883da11ab3fffff', '883da118d9fffff', '883da118d5fffff', '883da1036dfffff', '883da10369fffff', '883da11aadfffff', '883da1188dfffff', '883da118e9fffff', '883da118dbfffff', '883da10365fffff', '883da11a85fffff', '883da1189dfffff', '883da11aa9fffff', '883da11813fffff', '883da11881fffff', '883da118ebfffff', '883da11a81fffff', '883da118e3fffff', '883da10325fffff', '883da1185bfffff', '883da11a99fffff', '883da11a91fffff', '883da111a1fffff', '883da11a9b

In [268]:
#Query data from order_logs_snapshot

q = """
select order_id, accept_to_pickup_distance, city_name, customer_location_hex_8, customer_location_latitude, 
customer_location_longitude, hhmmss, hour, quarter_hour, service_obj_service_name, time_bucket, weekday, yyyymmdd,
order_status, cancel_reason, eta, map_riders, pickup_location_hex_8, captain_id, event_type, drop_location_hex_8, estimated_accept_to_pickup_distance,
updated_epoch
from orders.order_logs_immutable
where yyyymmdd = '20230517'
and city_name in ('Delhi')
and service_obj_service_name = 'Link'
and hour in ('08', '09', '10', '11')
and event_type in ('accepted', 'rider_busy', 'rider_reject')
"""


#Load data into pandas table
df_copy = pd.read_sql(q, connection)

In [269]:
df = df_copy.copy()
df.head()

,order_id,accept_to_pickup_distance,city_name,customer_location_hex_8,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,eta,map_riders,pickup_location_hex_8,captain_id,event_type,drop_location_hex_8,estimated_accept_to_pickup_distance,updated_epoch
0,646455c8850c6f6b8301f3ba,6.581,Delhi,883da11aa9fffff,28.448662,77.052940,094920,09,0945,Link,09:59,3,20230517,new,Long Pickup,15.0,"[""5d64a273d0286d106d665511"",""5c696cf5f2edc7336...",883da11aa9fffff,5d64a273d0286d106d665511,rider_reject,883da1036dfffff,1301.2,1684297252005
1,6464560ab7137846c0355658,NaN,Delhi,883da10349fffff,28.490017,77.105830,095026,09,0945,Link,09:59,3,20230517,new,None,4.0,"[""5e1e78f601b43542cd78a8ec"",""6264bf9ee98a6440d...",883da111b5fffff,618c8f659c02e46aacf4dbbd,rider_reject,883da111a9fffff,1890.0,1684297267007
2,646456760c558a0728c0ae33,NaN,Delhi,883da11253fffff,28.631594,77.032379,095214,09,0945,Link,09:59,3,20230517,new,None,2.0,"[""61dc83bafb8e4dd0c7322cb4"",""5d583613bde10e0d5...",883da11253fffff,61dc83bafb8e4dd0c7322cb4,rider_reject,883da18f25fffff,1133.2,1684297340727
3,646457bbf1b4f95226b0f4c5,NaN,Delhi,883da10433fffff,28.524506,77.394531,095739,09,0945,Link,09:59,3,20230517,new,None,3.0,"[""6303964fb1cde212b27032e8"",""608640f259edd80d9...",883da10453fffff,62ca74096803b113e676aaa9,rider_busy,883da106a5fffff,1958.6,1684297799748
4,646457de042e4f338d392c1b,NaN,Delhi,883da1a901fffff,28.635258,77.364349,095814,09,0945,Link,09:59,3,20230517,new,None,2.0,"[""61c868325379c0df814f801b"",""5c755b815e042733c...",883da1a901fffff,631db4b5bc0773d9d6c6255c,rider_busy,883da1a92bfffff,814.4,1684297768684


In [270]:
df[df['captain_id'] == '59b153ef0c3e1ca306385c51']['order_id'].nunique()

174

In [238]:
df['map_riders'] = df['map_riders'].apply(lambda x: json.loads(x))

In [239]:
df['mc_captain_1'] = df['map_riders'].apply(lambda x: '59b153ef0c3e1ca306385c51' in x)

In [240]:
first_captain_orders = df[df['mc_captain_1'] == True]['order_id'].unique().tolist()

In [241]:
print(first_captain_orders)

['646446954bcb4717d7737863', '6464479409b52e09c0d674f0', '6464486cee3bca5ddfa728cc', '646447a14bcb4717d7737aa6', '646455a97637581de5ded9a4', '646456e4ac1b992d0d7c5473', '646450f3dc7d445e1ecce63b', '646455a1b855c04242b16d3c', '6464564a739cd95b73ae914c', '64644e584bcb4717d77387b2', '64645653f1b4f95226b0f1f3', '64645167a5c51b1fbd45b3e7', '6464564509b52e09c0d69261', '646452a8eb7008132e52fcd6', '646454860c558a0728c0aa47', '6464548999b931616ed2790d', '6464581e4bcb4717d7739ca8', '64644cf8ee3bca5ddfa73122', '6464501e850c6f6b8301e73e', '6464516b850c6f6b8301ea5f', '64644fc92463953d1031f3b0', '6464590f7637581de5dee080', '64645c55e2baa269a5157647', '6464487609b52e09c0d6768e', '6464505edc7d445e1ecce4d1', '646456998f56e67633f95437', '646457f0739cd95b73ae94bc', '646451699ef2ba1ed0e7db31', '6464558fcc08b20893124024', '64644fd08b31b21a90c878c4', '64644bd42463953d1031eb6d', '646450c1dc7d445e1ecce5c5', '64645c1164adf3485d68873a', '646458bb8b31b21a90c88bdb', '646451ae8f56e67633f94a16', '64644c0df1b4f95226

In [242]:
print(len(first_captain_orders))

460


In [243]:
#Query data from order_logs_snapshot

q = """
select * from g2n.log_dispatch_prioritiser
where yyyymmdd = '20230517'
and service_detail_id = '5da4660028af187d8d52cc3c'
and hh in ('08', '09', '10')
and order_id in ('646446954bcb4717d7737863', '6464479409b52e09c0d674f0', '6464486cee3bca5ddfa728cc', '646447a14bcb4717d7737aa6', '646455a97637581de5ded9a4', '646456e4ac1b992d0d7c5473', '646450f3dc7d445e1ecce63b', '646455a1b855c04242b16d3c', '6464564a739cd95b73ae914c', '64644e584bcb4717d77387b2', '64645653f1b4f95226b0f1f3', '64645167a5c51b1fbd45b3e7', '6464564509b52e09c0d69261', '646452a8eb7008132e52fcd6', '646454860c558a0728c0aa47', '6464548999b931616ed2790d', '6464581e4bcb4717d7739ca8', '64644cf8ee3bca5ddfa73122', '6464501e850c6f6b8301e73e', '6464516b850c6f6b8301ea5f', '64644fc92463953d1031f3b0', '6464590f7637581de5dee080', '64645c55e2baa269a5157647', '6464487609b52e09c0d6768e', '6464505edc7d445e1ecce4d1', '646456998f56e67633f95437', '646457f0739cd95b73ae94bc', '646451699ef2ba1ed0e7db31', '6464558fcc08b20893124024', '64644fd08b31b21a90c878c4', '64644bd42463953d1031eb6d', '646450c1dc7d445e1ecce5c5', '64645c1164adf3485d68873a', '646458bb8b31b21a90c88bdb', '646451ae8f56e67633f94a16', '64644c0df1b4f95226b0dbc2', '646451be6e5ad378c185c6ed', '6464564463c7096e236aadce', '64644ff0a167ef4c66f8b971', '646460332463953d10321566', '64644c632463953d1031ec71', '6464503dcc08b20893123523', '6464504bee3bca5ddfa73852', '64645132cc08b2089312373a', '6464503fb855c04242b161d6', '64644cbaae01961a257aaa89', '6464560e8b31b21a90c88659', '64645ce0f5dab6510e04771f', '64645ca0cc08b20893124e56', '64645d19dc7d445e1eccfe01', '646458b764adf3485d68807e', '646458c7b855c04242b17375', '64645c0a0c558a0728c0b921', '6464617599b931616ed29279', '64644fefb7137846c0354932', '646452872463953d1031f9ef', '646456bff1b4f95226b0f2c2', '64645195a5c51b1fbd45b466', '64645658cc08b208931241b6', '64644b478f56e67633f93c60', '6464614ba167ef4c66f8dbba', '64644b448b31b21a90c86ecc', '64644fc009b52e09c0d684bd', '6464531a4bcb4717d773927a', '6464561df1b4f95226b0f195', '646451318d4ebf35608b7a98', '6464515663c7096e236aa370', '6464556e63c7096e236aac1b', '64645754b7137846c03558ca', '64644e469ef2ba1ed0e7d41b', '64645412042e4f338d39247a', '64644c9b4bcb4717d77383eb', '64645ba7b855c04242b178e7', '6464609564adf3485d688fc7', '6464512ee2baa269a5155f6b', '64644f6ccc08b2089312335c', '646458980c558a0728c0b263', '646458f564adf3485d6880ef', '646453b28d4ebf35608b7fff', '646450aa2463953d1031f5b8', '64645843cc08b20893124573', '646454c8cc08b20893123ea2', '646447952463953d1031e30c', '646454df09b52e09c0d68f79', '64644b1b6e5ad378c185b8ab', '646457700c558a0728c0b034', '64645a25a5c51b1fbd45c6b7', '646461908d4ebf35608b9b01', '64645973b7137846c0355d60', '646459750c558a0728c0b407', '646447d78b31b21a90c86894', '64644c62e2baa269a51554e6', '6464526f8b31b21a90c87eb2', '6464558509b52e09c0d690dc', '64644d66850c6f6b8301e11a', '6464501e9c55e17ceea77e2d', '646450b8b855c04242b16304', '64645bfea167ef4c66f8d195', '64645bcdf1b4f95226b0fcb2', '646461dfeb7008132e531acb', '64646043ee3bca5ddfa7588b', '6464611e4bcb4717d773adfd', '646458616e5ad378c185d453', '646446c4cc08b20893122337', '646447f2f1b4f95226b0d424', '6464525c8b31b21a90c87e78', '646451edf5dab6510e046120', '6464523bae01961a257ab67b', '64644f8eb855c04242b16032', '6464606c4bcb4717d773aca2', '64645cdfb855c04242b17b56', '6464598b8b31b21a90c88dac', '64645be7b855c04242b1796f', '64645875850c6f6b8301f911', '6464585a850c6f6b8301f8c9', '6464603b99b931616ed29013', '646459f1042e4f338d39307f', '64644d3e2463953d1031ee2c', '64644e3f09b52e09c0d68149', '646457d863c7096e236ab121', '6464566fa5c51b1fbd45beca', '646455d0f1b4f95226b0f100', '64645096b855c04242b162b6', '64645132ee3bca5ddfa73a75', '64644fafa167ef4c66f8b8f2', '646455a7850c6f6b8301f372', '64645a42850c6f6b8301fc9f', '64644db7b7137846c035443f', '6464550ddc7d445e1ecceeb3', '64644c7b2463953d1031ecab', '64644c2d8f56e67633f93e29', '646447f39ef2ba1ed0e7c80f', '64645f909ef2ba1ed0e7f74c', '64645cb9042e4f338d3935d4', '64645a7ef1b4f95226b0fa2f', '646460f5ee3bca5ddfa759d2', '646459c22463953d10320905', '64645be3b855c04242b17964', '646450599ef2ba1ed0e7d8c0', '64645136a5c51b1fbd45b373', '646455cce2baa269a5156931', '64644cbb6e5ad378c185bbee', '6464508aae01961a257ab2be', '646450e663c7096e236aa281', '646453cf2463953d1031fc89', '646453c72463953d1031fc77', '646456158b31b21a90c8866b', '646450d0ac1b992d0d7c484d', '64645a6d042e4f338d393177', '64645c488b31b21a90c89301', '646451e0f1b4f95226b0e8e3', '6464475809b52e09c0d67479', '6464487be2baa269a5154d81', '64644f9e63c7096e236a9f60', '646454c46e5ad378c185cd2c', '64644b10b855c04242b156ed', '64644b24ee3bca5ddfa72dab', '646458b80c558a0728c0b29d', '646459b92463953d103208fa', '646461c6e2baa269a5158070', '6464588bf5dab6510e046eb6', '64645108850c6f6b8301e989', '64644db863c7096e236a9af7', '646456baac1b992d0d7c5413', '6464584763c7096e236ab213', '64645985b855c04242b174c7', '646459e799b931616ed283e3', '64645a56a5c51b1fbd45c71e', '646458b28d4ebf35608b8a03', '64644bf98b31b21a90c87032', '64644ad0850c6f6b8301dbc2', '64644b19ae01961a257aa778', '64645057e2baa269a5155d81', '6464494909b52e09c0d677ee', '646446628f56e67633f9337c', '64645a6b99b931616ed284f1', '64645a007637581de5dee29d', '646461b89ef2ba1ed0e7fb2d', '64645b7ccc08b20893124c17', '6464603bf1b4f95226b10508', '6464611d7637581de5deefdb', '6464592cb7137846c0355cd5', '646458efcc08b208931246ea', '64645773b7137846c035590d', '646451322463953d1031f712', '64644ce1e2baa269a51555dc', '646450afb7137846c0354b0e', '64645adc8f56e67633f95c56', '64645b156e5ad378c185d9a4', '6464552fa167ef4c66f8c424', '646451dd63c7096e236aa496', '64644f80b855c04242b16016', '64645842b7137846c0355adb', '64645cb8042e4f338d3935d2', '64645cd40c558a0728c0ba88', '646458ad64adf3485d688069', '64646110ee3bca5ddfa75a01', '64644f618b31b21a90c877a8', '6464558e8f56e67633f9523f', '6464576c6e5ad378c185d263', '64644af2a5c51b1fbd45a585', '64645592b855c04242b16d1a', '6464553f0c558a0728c0abce', '646451b0f5dab6510e04609c', '646451c5dc7d445e1ecce7e8', '64645a68dc7d445e1eccf901', '6464584ce2baa269a5156e37', '64645c56b7137846c03562b9', '64645a70b7137846c0355f4a', '64645a01739cd95b73ae9924', '64645b45cc08b20893124ba0', '64645bbc63c7096e236ab8b6', '646460caeb7008132e5318e2', '646455ccb855c04242b16d94', '64645679cc08b20893124204', '64645071a167ef4c66f8ba76', '646452af9ef2ba1ed0e7ddf9', '6464488299b931616ed2608a', '64644dd9850c6f6b8301e218', '64645683dc7d445e1eccf169', '646456f109b52e09c0d69396', '64645057e2baa269a5155d80', '64644d8c6e5ad378c185bd9e', '64645757dc7d445e1eccf2ef', '64644b6bdc7d445e1eccda26', '646453f6850c6f6b8301effe', '64645723739cd95b73ae9306', '646458a4dc7d445e1eccf5a1', '64645875f1b4f95226b0f664', '64645af7042e4f338d393274', '646460aacc08b208931255ca', '646458969ef2ba1ed0e7ea48', '646457c4042e4f338d392bd8', '6464577b64adf3485d687dca', '64645877eb7008132e5308b8', '64645a2e7637581de5dee2fd', '64645aa38b31b21a90c88ffd', '646461092463953d103216f6', '646450e799b931616ed271a8', '6464548ea5c51b1fbd45baf2', '64644e364bcb4717d773876b', '64644faf4bcb4717d7738aa8', '6464548d8b31b21a90c8835a', '64644d669c55e17ceea77823', '64644e0ab7137846c0354500', '646451548f56e67633f94956', '64644de3850c6f6b8301e22f', '646454eb4bcb4717d7739636', '6464502dcc08b208931234f7', '64644899a167ef4c66f8aae2', '646452038b31b21a90c87dac', '64644780a5c51b1fbd459f33', '6464591c64adf3485d688141', '6464512ab855c04242b163fc', '6464530ccc08b20893123b18', '646455d909b52e09c0d69189', '646461dff5dab6510e0480ee', '64644cc98f56e67633f93f5f', '64644c2199b931616ed266f3', '646450854bcb4717d7738cc8', '64644d0f9c55e17ceea77776', '64644fceb7137846c03548ea', '646457cd99b931616ed27f7d', '646451b29ef2ba1ed0e7dbd5', '646450139c55e17ceea77e0b', '646458377637581de5dedecc', '64644bc2dc7d445e1eccdacf', '64645aaa6e5ad378c185d8da', '64645cbe0c558a0728c0ba68', '64645b7bf1b4f95226b0fc14', '64645580cc08b20893124000', '6464559f99b931616ed27b3b', '64644bdca167ef4c66f8b0e4', '64644bb68f56e67633f93d37', '64644d10a167ef4c66f8b343', '64645005a5c51b1fbd45b0a8', '64645150ee3bca5ddfa73abc', '646452ac0c558a0728c0a68c', '64644dcf9c55e17ceea77900', '64645529042e4f338d3926bb', '646451a0ae01961a257ab53a', '64644d928b31b21a90c87357', '646446dcae01961a257a9fd5', '64646114eb7008132e53196a', '64644720e2baa269a5154b0d', '646454e9dc7d445e1eccee69', '6464553d739cd95b73ae8f2e', '64644d544bcb4717d7738578', '64644d6f99b931616ed269a4', '646454dfb7137846c03553e1', '64644dd09c55e17ceea77903', '646461c664adf3485d6891df', '646454aa0c558a0728c0aa91', '64644baadc7d445e1eccdaa1', '64644f6f4bcb4717d7738a19', '646457e2ee3bca5ddfa747fd', '646455e3e2baa269a5156963', '64645162a5c51b1fbd45b3dd', '6464586d042e4f338d392d5a', '646450e9dc7d445e1ecce628', '6464517f8b31b21a90c87c8b', '646454567637581de5ded713', '64645617f1b4f95226b0f188', '646452ad4bcb4717d7739175', '6464509eb855c04242b162c8', '64644ce4850c6f6b8301dff9', '64645c13cc08b20893124d4b', '64644f7ccc08b20893123384', '64644e369ef2ba1ed0e7d3f0', '646451590c558a0728c0a3ae', '646460aba5c51b1fbd45d32e', '646455baee3bca5ddfa743c4', '6464553fae01961a257abcd8', '646454d0b855c04242b16b96', '64644a9e2463953d1031e8e4', '64644d8a850c6f6b8301e161', '646456fbee3bca5ddfa74620', '64645af32463953d10320b5c', '64645410b7137846c0355228', '646454fc042e4f338d392653', '646449ae8b31b21a90c86bb6', '64645cbff5dab6510e0476e3', '646454a8ac1b992d0d7c5011', '64644c334bcb4717d773832d', '646454697637581de5ded73e', '64644d5ba167ef4c66f8b3e8', '64644fe1dc7d445e1ecce3a7', '6464471f4bcb4717d773797c', '646454dd042e4f338d392614', '64645053f1b4f95226b0e54a', '64645483b7137846c0355325', '646450089c55e17ceea77df4', '64644fed99b931616ed26f46', '64644b98dc7d445e1eccda7d', '64644ad1850c6f6b8301dbc6', '64645004cc08b208931234a6', '64644f92f1b4f95226b0e372', '646455adeb7008132e530336', '64644aef2463953d1031e97f', '6464566db7137846c0355715', '646458778f56e67633f957d6', '64644cc59ef2ba1ed0e7d0ea', '64644d31ee3bca5ddfa73190', '64644b8ecc08b20893122b9e', '64644a5163c7096e236a9403', '6464540864adf3485d6876cc', '646451f409b52e09c0d68973', '64644b622463953d1031ea69', '64644e1e63c7096e236a9bce', '6464516ce2baa269a5155ff9', '646455968d4ebf35608b83d6', '64645c6c850c6f6b830200d2', '646458b864adf3485d688080', '6464476f09b52e09c0d674a1', '6464503109b52e09c0d685c9', '646458c9b7137846c0355c11', '646448b48d4ebf35608b6943', '64645ca509b52e09c0d69f07', '64645c36ee3bca5ddfa750a9', '646447516e5ad378c185b1f5', '64644c8ae2baa269a5155530', '64644bd9ee3bca5ddfa72f0a', '64644da109b52e09c0d6800c', '64644fb199b931616ed26ea8', '64644e1d2463953d1031f00f', '646457a06e5ad378c185d2d8', '64645c32f5dab6510e0475e8', '64645473ae01961a257abb47', '646457b5a167ef4c66f8c968', '64645238a167ef4c66f8be54', '64644b35ee3bca5ddfa72dcb', '646456e49ef2ba1ed0e7e6ab', '646447b7a5c51b1fbd459fa9', '64644860dc7d445e1eccd4fc', '64645251b7137846c0354e83', '646450048d4ebf35608b7803', '6464567f4bcb4717d773996d', '646458358b31b21a90c88aaf', '646458ad8d4ebf35608b89f8', '64645bcd8f56e67633f95dfb', '64645757f1b4f95226b0f3eb', '6464500de2baa269a5155cc0', '646448a3b855c04242b1529c', '64645743a167ef4c66f8c86c', '64645a9eeb7008132e530d22', '64646176f5dab6510e048038', '64644be9ee3bca5ddfa72f26', '646459cd4bcb4717d773a012', '646460c863c7096e236ac20a', '646447e4e2baa269a5154c83', '6464537a9ef2ba1ed0e7dfaf', '646455cfcc08b208931240a4', '64645c326e5ad378c185dbe8', '64644d24f1b4f95226b0de0f', '64645506042e4f338d392668', '64645d26b855c04242b17bd9', '6464586d850c6f6b8301f8fa', '64644fc28d4ebf35608b7762', '6464592adc7d445e1eccf6aa', '6464561e850c6f6b8301f466', '64645166dc7d445e1ecce713', '64645637f5dab6510e0469ec', '64644b0663c7096e236a9557', '646454b38f56e67633f95062', '6464503eac1b992d0d7c4706', '646455790c558a0728c0ac49', '64644c4e09b52e09c0d67d81', '646457b4dc7d445e1eccf3ba', '64644895ae01961a257aa31d', '6464547f8d4ebf35608b81b6', '64645952b855c04242b17471', '64645be364adf3485d6886d1', '64646159eb7008132e5319e4', '64645033b855c04242b161ba', '646451e1dc7d445e1ecce820', '646452a2eb7008132e52fcc6', '64644b9cae01961a257aa854', '6464585109b52e09c0d69679', '64645bf08b31b21a90c89269', '646453ae63c7096e236aa8a2', '6464542f042e4f338d3924b8', '64644dd78f56e67633f9419e', '64645f9263c7096e236abfb3', '6464577bb7137846c035591e', '64644b38b855c04242b15745', '6464473f63c7096e236a8e6c', '64645798850c6f6b8301f725', '64645bbb9ef2ba1ed0e7f034', '646455ebee3bca5ddfa7441b', '646457878f56e67633f955f5', '646460368f56e67633f96606', '64645cdacc08b20893124eb8', '646457d88d4ebf35608b8853', '6464522b99b931616ed27477', '646454f7739cd95b73ae8ea7', '64644b61b855c04242b15785', '6464472edc7d445e1eccd2be', '64644c4bae01961a257aa99f', '6464618bee3bca5ddfa75ad7', '64645ca4ee3bca5ddfa75168', '64645bf30c558a0728c0b8ef', '64645aa7cc08b20893124a71', '64644db7ac1b992d0d7c4162', '6464484c09b52e09c0d67651', '64644690ee3bca5ddfa72547', '646447feac1b992d0d7c3692', '64645622f1b4f95226b0f199', '6464511c2463953d1031f6dc', '6464511de2baa269a5155f52', '64644c4809b52e09c0d67d75', '646460ba8b31b21a90c89bab', '64645c2be2baa269a51575f4', '64645c3ceb7008132e531046', '6464465bac1b992d0d7c335f', '6464566d8d4ebf35608b8588', '64645905b7137846c0355c83', '646456b7a167ef4c66f8c74c', '646450434bcb4717d7738c28', '64644fadb7137846c035489f', '64644e6209b52e09c0d68192')
"""


#Load data into pandas table
df_dispatch_copy2 = pd.read_sql(q, connection)

In [244]:
df_dispatch_copy2

,created_on,event_id,updated_at,version,enabled_captain_filter_types,order_id,compositeHexId,prioritization_stage,enabled_strategies_with_config,filtered_captains,captains,city_id,customer_id,drop_location,city_name,correlation,last_mile_distance,order_type,payment_type,pickup_location,service_category,service_detail_id,retry_count,captains_with_score,enabled_strategies_with_scores,epkm,infeasible_captains,prioritized_captains,yyyymmdd,hh,quarter_hour
0,1684297760881,eea63233-f59b-123d-adc0-18d0eb0338a9,1684297760881,1.3.0,"[""economicFeasibilityOfARide"",""failedMatchInTh...",646457f0739cd95b73ae94bc,,dispatch,"{""first_mile_distance"":{""bucket"":{""params"":[{""...",[],"[{""firstMileDistance"":2047.6,""id"":""634fd5961d7...",5bc5ac2312477c2ece769591,63cb741319eefc589d830bb0,"{""lat"":28.459147,""lng"":77.072858}",Delhi,"{""id"":""646457f0739cd95b73ae94bc"",""type"":""order""}",2940.0,app,cash,"{""lat"":28.462905220240838,""lng"":77.05994881689...",app,5da4660028af187d8d52cc3c,0,"[{""captain_id"":""637ed7b61ed9b803c8b5b1b6"",""cap...",None,None,None,None,20230517,09,0945
1,1684295184781,60822dae-0cea-6d51-cac2-e42d332ef729,1684295184781,1.3.0,"[""economicFeasibilityOfARide"",""failedMatchInTh...",64644dd9850c6f6b8301e218,,dispatch,"{""first_mile_distance"":{""bucket"":{""params"":[{""...","[{""filter"":[{""meta"":{""earnings_per_km"":{""epkm""...","[{""earnings"":{""amount"":45.96,""preCommissionAmo...",5bc5ac2312477c2ece769591,5cec138d25ee3218d4e54d0d,"{""lat"":28.4459206,""lng"":77.039982}",Delhi,"{""id"":""64644dd9850c6f6b8301e218"",""type"":""order""}",5580.0,app,upi,"{""lat"":28.476886641515705,""lng"":77.06634219735...",app,5da4660028af187d8d52cc3c,0,"[{""captain_id"":""633807df1f3de757c23f6311"",""cap...",None,None,None,None,20230517,09,0915
2,1684296025665,546b1d3b-c43a-cfe2-b289-b910d2db9b91,1684296025665,1.3.0,"[""economicFeasibilityOfARide"",""failedMatchInTh...",646451590c558a0728c0a3ae,,dispatch,"{""first_mile_distance"":{""bucket"":{""params"":[{""...",[],"[{""earnings"":{""amount"":71.45,""preCommissionAmo...",5bc5ac2312477c2ece769591,5f38c37f44a784655bb0c736,"{""lat"":28.4346993,""lng"":77.0430483}",Delhi,"{""id"":""646451590c558a0728c0a3ae"",""type"":""order""}",7390.0,app,qrcode,"{""lat"":28.4730752,""lng"":77.0857256}",app,5da4660028af187d8d52cc3c,0,"[{""captain_id"":""626dd3cce98a6444b59abd89"",""cap...",None,None,None,None,20230517,09,0915
3,1684296077036,829c82b3-6a4d-8174-77f5-d3b9298dfe97,1684296077036,1.3.0,"[""economicFeasibilityOfARide"",""failedMatchInTh...",64645150ee3bca5ddfa73abc,,dispatch,"{""first_mile_distance"":{""bucket"":{""params"":[{""...","[{""filter"":[{""meta"":{""earnings_per_km"":{""epkm""...","[{""earnings"":{""amount"":41.06,""preCommissionAmo...",5bc5ac2312477c2ece769591,5c7d4b61875c985362612c3b,"{""lat"":28.446630824993463,""lng"":77.10062351077...",Delhi,"{""id"":""64645150ee3bca5ddfa73abc"",""type"":""order""}",3740.0,app,paytm,"{""lat"":28.45570718232541,""lng"":77.08533927798271}",app,5da4660028af187d8d52cc3c,0,"[{""captain_id"":""641e98636c852b89c37f79e0"",""cap...",None,None,None,None,20230517,09,0930
4,1684296091732,2cfc69c7-9428-789c-cc5e-0806fc222b8a,1684296091732,1.3.0,"[""economicFeasibilityOfARide"",""failedMatchInTh...",6464516b850c6f6b8301ea5f,,dispatch,"{""first_mile_distance"":{""bucket"":{""params"":[{""...",[],"[{""earnings"":{""amount"":38.25,""preCommissionAmo...",5bc5ac2312477c2ece769591,6304e193fd2bff14c8b25fe2,"{""lat"":28.4526542,""lng"":77.0685665}",Delhi,"{""id"":""6464516b850c6f6b8301ea5f"",""type"":""order""}",2660.0,app,cash,"{""lat"":28.45907517050111,""lng"":77.08668742328882}",app,5da4660028af187d8d52cc3c,0,"[{""captain_id"":""6183dc2257f14436adb45173"",""cap...",None,None,None,None,20230517,09,0930
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
532,1684294028351,cb96b9b5-9973-9d2a-7407-a0ab9b83e1b3,1684294028351,1.3.0,"[""economicFeasibilityOfARide"",""failedMatchIn

In [257]:
order_mismatch = []
sup_pri_orders = df_dispatch_copy2['order_id'].unique().tolist()
for order in first_captain_orders:
    if order not in sup_pri_orders:
        order_mismatch.append(order)

In [265]:
df_filter = df[(df['captain_id'] == '59b153ef0c3e1ca306385c51') & (df['order_id'].isin(order_mismatch))]
df_filter['map_riders'] = df_filter['map_riders'].apply(lambda x: json.loads(x))
df_filter['captain_index'] = df_filter['map_riders'].apply(lambda x: x.index('59b153ef0c3e1ca306385c51'))
df_filter['captain_index'].value_counts()

captain_index
0    115
1      1
Name: count, dtype: int64

In [271]:
df_filter2 = df[df['order_id'].isin(order_mismatch)]
df_filter2 = df_filter2.sort_values(by = 'updated_epoch')

In [274]:
x = df_filter2[['order_id', 'estimated_accept_to_pickup_distance', 'captain_id', 'event_type']].head(20)
x[x['order_id'].isin(['646446628f56e67633f9337c', '646447e4e2baa269a5154c83', '6464479409b52e09c0d674f0'])]

,order_id,estimated_accept_to_pickup_distance,captain_id,event_type
7344,646446628f56e67633f9337c,1163.0,59b153ef0c3e1ca306385c51,rider_reject
3853,646446628f56e67633f9337c,15.9,5bd1495b49f94601ccfe176e,accepted
14321,6464479409b52e09c0d674f0,918.3,59b153ef0c3e1ca306385c51,rider_reject
42047,6464479409b52e09c0d674f0,0.0,63f2f6cee2e2273548ce2556,rider_busy
154721,6464479409b52e09c0d674f0,63.0,631d44c50ced23df05c043af,rider_busy
66968,6464479409b52e09c0d674f0,59.9,631ad203e21090629210cfed,accepted
154985,646447e4e2baa269a5154c83,239.6,59b153ef0c3e1ca306385c51,rider_reject
85799,646447e4e2baa269a5154c83,36.9,5ef83912f540b8eec7ad63f2,accepted


In [275]:
print(order_mismatch)

['6464479409b52e09c0d674f0', '646447a14bcb4717d7737aa6', '6464564a739cd95b73ae914c', '64645653f1b4f95226b0f1f3', '6464564509b52e09c0d69261', '64644cf8ee3bca5ddfa73122', '6464590f7637581de5dee080', '6464558fcc08b20893124024', '646450c1dc7d445e1ecce5c5', '646458bb8b31b21a90c88bdb', '64644c0df1b4f95226b0dbc2', '6464564463c7096e236aadce', '64644ff0a167ef4c66f8b971', '6464504bee3bca5ddfa73852', '64644cbaae01961a257aaa89', '64645ce0f5dab6510e04771f', '6464617599b931616ed29279', '64645195a5c51b1fbd45b466', '64644b478f56e67633f93c60', '64644e469ef2ba1ed0e7d41b', '64644c9b4bcb4717d77383eb', '646458980c558a0728c0b263', '646458f564adf3485d6880ef', '646447952463953d1031e30c', '64644b1b6e5ad378c185b8ab', '646461908d4ebf35608b9b01', '646459750c558a0728c0b407', '64644c62e2baa269a51554e6', '6464558509b52e09c0d690dc', '64644d66850c6f6b8301e11a', '6464501e9c55e17ceea77e2d', '646450b8b855c04242b16304', '64645bfea167ef4c66f8d195', '64645bcdf1b4f95226b0fcb2', '64646043ee3bca5ddfa7588b', '646446c4cc08b20893

In [276]:
print(len(order_mismatch))

129


In [281]:
#Query data from order_logs_snapshot

q = """
select * from g2n.log_dispatch_prioritiser
where yyyymmdd = '20230517'
and order_id in ('64644782850c6f6b8301d5c2')
"""


#Load data into pandas table
df_dispatch_copy2 = pd.read_sql(q, connection)

In [282]:
df_dispatch_copy2

,created_on,event_id,updated_at,version,enabled_captain_filter_types,order_id,compositeHexId,prioritization_stage,enabled_strategies_with_config,filtered_captains,captains,city_id,customer_id,drop_location,city_name,correlation,last_mile_distance,order_type,payment_type,pickup_location,service_category,service_detail_id,retry_count,captains_with_score,enabled_strategies_with_scores,epkm,infeasible_captains,prioritized_captains,yyyymmdd,hh,quarter_hour
0,1684293507270,036e4eaa-3648-8da8-35fd-b4702b2ab0c8,1684293507270,1.3.0,"[""economicFeasibilityOfARide""]",64644782850c6f6b8301d5c2,,dispatch,"{""first_mile_distance"":{""bucket"":{""params"":[{""...",[],"[{""earnings"":{""amount"":35.0,""preCommissionAmou...",5bc5ad5f12477c2ece7695a3,62c0f973d9dd89d00070f609,"{""lat"":18.5784239,""lng"":73.737183}",Pune,"{""id"":""64644782850c6f6b8301d5c2"",""type"":""order""}",1890.0,auto,cash,"{""lat"":18.58548890931908,""lng"":73.72799847275019}",auto,61769b50d21e19252c6f60e8,0,"[{""captain_id"":""62aab722d3109e8a0b3a2e14"",""cap...",None,None,None,None,20230517,08,0845
1,1684293639872,bd6b3179-0181-d591-ed92-c4b5ddcab561,1684293639872,1.3.0,"[""economicFeasibilityOfARide""]",64644782850c6f6b8301d5c2,,dispatch,"{""first_mile_distance"":{""bucket"":{""params"":[{""...",[],"[{""earnings"":{""amount"":35.0,""preCommissionAmou...",5bc5ad5f12477c2ece7695a3,62c0f973d9dd89d00070f609,"{""lat"":18.5784239,""lng"":73.737183}",Pune,"{""id"":""64644782850c6f6b8301d5c2"",""type"":""order""}",1890.0,auto,cash,"{""lat"":18.58548890931908,""lng"":73.72799847275019}",auto,61769b50d21e19252c6f60e8,0,"[{""captain_id"":""619fb390544fd85e84bbf3d4"",""cap...",None,None,None,None,20230517,08,0845


In [266]:
df_filter['order_id'].nunique()

112

In [262]:
print(len(order_mismatch))

129


In [247]:
df_dispatch_copy2['captains_with_score'] = df_dispatch_copy2['captains_with_score'].apply(lambda x: json.loads(x))

In [248]:
fm_distance = []
captain_rank = []
micro_corridor = []
location = []
captain_list = df_dispatch_copy2['captains_with_score'].values
for captains in captain_list:
    for captain in captains:
        if captain['captain_id'] == '59b153ef0c3e1ca306385c51':
            fm_distance.append(captain['first_mile_distance'])
            captain_rank.append(captains.index(captain))
            micro_corridor.append(captain['score']['micro_corridor'])
            

In [249]:
print(fm_distance)

[2338.1, 3032.7, 2949.1, 2765.7, 4638.5, 2951.9, 2797.7, 2948.9, 3057.2, 2017.7, 2706.7, 3241.4, 2912.4, 3146.1, 3384.6, 4350.6, 2936.7, 3572.9, 2658.0, 2949.2, 3929.5, 3141.9, 3714.5, 3318.8, 2779.5, 3306.6, 2014.7, 3484.6, 3307.0, 3352.0, 3418.1, 4597.6, 3738.5, 3947.9, 3092.7, 3468.5, 3134.3, 2470.3, 3094.4, 3271.9, 3556.4, 2955.8, 2209.6, 4035.9, 2292.5, 3475.6, 3437.0, 2062.4, 2988.6, 3342.5, 2541.3, 3133.1, 3584.8, 2986.7, 2832.2, 2834.7, 3485.6, 2022.5, 3087.6, 3314.4, 3492.2, 3158.7, 2016.8, 2950.1, 2389.3, 2629.3, 3197.9, 2867.4, 4674.2, 3321.9, 3143.9, 2983.5, 3138.5, 3162.1, 3133.2, 3016.3, 3962.5, 3436.4, 3379.6, 2904.5, 3454.2, 2771.8, 3612.0, 2742.6, 2753.1, 2826.3, 3493.6, 2121.7, 2726.0, 3081.2, 3122.7, 2895.7, 3033.0, 4560.4, 4553.7, 3212.2, 3040.6, 3192.3, 2630.8, 2556.7, 2633.7, 2542.7, 2034.1, 2904.8, 2886.9, 3220.7, 3872.5, 2072.3, 2649.6, 3812.8, 3144.4, 2533.5, 3008.7, 3743.4, 2538.4, 3022.8, 2058.6, 2087.4, 2791.5, 3098.4, 2542.7, 2905.2, 2828.7, 2030.6, 3296.6,

In [253]:
for x in fm_distance:
    if x < 2000:
        print(x)

In [252]:
print(micro_corridor)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [174]:
print(captain_rank)

[43, 19, 27, 7, 41, 20, 36, 4, 26, 24, 12, 11, 36, 61, 24, 21, 36, 54, 19, 2, 57, 19, 6, 97, 19, 37, 24, 12, 14, 63, 6, 6, 7, 7, 84, 81, 16, 4, 8, 11, 50, 74, 4, 5, 76, 46, 6, 43, 10, 63, 9, 12, 7, 6, 35, 5, 8, 12, 61, 14, 9, 3, 22, 5, 3, 3, 12, 3, 5, 16, 4, 22, 7, 1, 12, 6, 3, 7, 11, 5, 12, 7, 4, 11, 4, 1, 4, 6, 2, 3, 2, 4, 127, 3, 8, 6, 10, 3, 3, 17, 3, 2, 1, 7, 7, 5, 19, 45, 43, 10, 13, 10, 6, 5, 10, 14, 72, 76, 3, 4, 5, 10, 7, 3, 17, 10, 6, 6, 4, 6, 3, 12, 60, 20, 20, 39, 9, 4, 4, 13]


In [175]:
print(micro_corridor)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [42]:
#Query data from order_logs_snapshot

q = """
select * from g2n.log_dispatch_prioritiser
where yyyymmdd = '20230515'
and service_detail_id = '5da4660028af187d8d52cc3c'
and order_id in ('6461dd89780955423c323d29', '6461d9b297b0a30bbd66f587', '6461dccd8753985ab62d977e', '6461dbeb8753985ab62d9584', '6461db90b97fe77d4ecf91b7',
 '6461d74e751ce46ce048ea48', '6461d7141e1f7e2e37653760', '6461dae5ca5873133f31596f', '6461d9938753985ab62d9080', '6461d8cf8956b6528798490c',
   '6461dccb780955423c323b80', '6461d9385ac5d8625a819c48', '6461dae5cd5d212b73a0eb1e', '6461da5c9c50b541b2df2630', '6461dcf643583753739896d8',
     '6461df76751ce46ce048fbb9', '6461d5bd5ac5d8625a819502', '6461d754780955423c322f8d', '6461ddcc8eac592e5cae9d4e', '6461dee44358375373989aed',
       '6461d823780955423c32315d', '6461da858eac592e5cae964c', '6461dad19f166a55ff604990', '6461dd0c9f166a55ff604e47', '6461dee4b97fe77d4ecf98c6',
         '6461d72d8753985ab62d8b4b', '6461d6ff43583753739889d9', '6461dbcfa78efb0c7dc72b91', '6461dc088eac592e5cae996a', '6461d64db97fe77d4ecf861b',
           '6461da69ca5873133f31585d', '6461d6b61e1f7e2e3765368a', '6461df4bf4a8ae312cd1f650', '6461dd2fa78efb0c7dc72e9a', '6461d8d39c50b541b2df22f7',
             '6461dbc09f166a55ff604ba6', '6461dd46cd5d212b73a0f01d', '6461dcaf8eac592e5cae9ab3', '6461db618eac592e5cae9809', '6461dbb91e1f7e2e37654165',
               '6461de1df4a8ae312cd1f39c', '6461d94f780955423c3233cd', '6461de08cd5d212b73a0f1cc', '6461dce9ad68a90a314dce9f', '6461de36751ce46ce048f8fe',
                 '6461d5fecd5d212b73a0e034', '6461d79eb97fe77d4ecf8910', '6461df5597b0a30bbd6701d5', '6461dde51e1f7e2e3765465f', '6461de5a5ac5d8625a81a759',
                   '6461d9dc1e1f7e2e37653d86', '6461d6f643583753739889c8', '6461d60c1e1f7e2e3765351d', '6461dc8943583753739895fc', '6461dedf9c50b541b2df2fc2',
                     '6461d59cf4a8ae312cd1e0f8', '6461da7a9c50b541b2df266b', '6461d5a1a78efb0c7dc71e6e', '6461d75d8753985ab62d8bb2', '6461d7ff612b71236703b50f',
                       '6461d9b9780955423c3234d6', '6461dc9b612b71236703bf5b', '6461dda697b0a30bbd66fe36', '6461de869c50b541b2df2f1c', '6461dd1aca5873133f315e4f',
                         '6461d82f5ac5d8625a819a3a', '6461d7c08956b652879846c7', '6461d62b612b71236703b103')
"""


#Load data into pandas table
df_dispatch_copy2 = pd.read_sql(q, connection)

In [215]:
#Query data from order_logs_snapshot

q = """
select * from g2n.log_dispatch_prioritiser
where yyyymmdd = '20230516'
and service_detail_id = '5da4660028af187d8d52cc3c'
and order_id = '6464617599b931616ed29279'
"""


#Load data into pandas table
df_dispatch_copy = pd.read_sql(q, connection)

In [216]:
df_dispatch_copy

,created_on,event_id,updated_at,version,enabled_captain_filter_types,order_id,compositeHexId,prioritization_stage,enabled_strategies_with_config,filtered_captains,captains,city_id,customer_id,drop_location,city_name,correlation,last_mile_distance,order_type,payment_type,pickup_location,service_category,service_detail_id,retry_count,captains_with_score,enabled_strategies_with_scores,epkm,infeasible_captains,prioritized_captains,yyyymmdd,hh,quarter_hour


In [146]:
df_dispatch2 = df_dispatch_copy2.copy()
df_dispatch2.head()

,created_on,event_id,updated_at,version,enabled_captain_filter_types,order_id,compositeHexId,prioritization_stage,enabled_strategies_with_config,filtered_captains,captains,city_id,customer_id,drop_location,city_name,correlation,last_mile_distance,order_type,payment_type,pickup_location,service_category,service_detail_id,retry_count,captains_with_score,enabled_strategies_with_scores,epkm,infeasible_captains,prioritized_captains,yyyymmdd,hh,quarter_hour
0,1684209910717,92980715-8bcb-7b68-910d-6545a8b97ab5,1684209910717,1.3.0,"[""economicFeasibilityOfARide"",""failedMatchInTh...",646300f69ad8e2253f17829c,,dispatch,"{""first_mile_distance"":{""bucket"":{""params"":[{""...",[],"[{""earnings"":{""amount"":53.87,""preCommissionAmo...",5bc5ac2312477c2ece769591,5d1580563b752c45cf8bce8e,"{""lat"":28.459147,""lng"":77.072858}",Delhi,"{""id"":""646300f69ad8e2253f17829c"",""type"":""order""}",4770.0,app,cash,"{""lat"":28.45348109269419,""lng"":77.09132214882767}",app,5da4660028af187d8d52cc3c,0,"[{""captain_id"":""644e0a66644efb1c5fee0bd9"",""cap...",None,None,None,None,20230516,09,0930
1,1684210168813,49842c6a-ac9d-3863-1d4b-1665bc8be8ec,1684210168813,1.3.0,"[""economicFeasibilityOfARide"",""failedMatchInTh...",64630197a5c51b1fbd43d008,,dispatch,"{""first_mile_distance"":{""bucket"":{""params"":[{""...",[],"[{""earnings"":{""amount"":95.02,""preCommissionAmo...",5bc5ac2312477c2ece769591,62bcd17417af9a5cd065dda8,"{""lat"":28.412011773817405,""lng"":77.10839923471...",Delhi,"{""id"":""64630197a5c51b1fbd43d008"",""type"":""order""}",9400.0,app,cash,"{""lat"":28.455322801568695,""lng"":77.08740960806...",app,5da4660028af187d8d52cc3c,0,"[{""captain_id"":""5fb55921e41162525b22df8f"",""cap...",None,None,None,None,20230516,09,0930
2,1684208237670,c5c24b54-5d38-335c-46fc-d76c6e08e6ea,1684208237670,1.3.0,"[""economicFeasibilityOfARide"",""failedMatchInTh...",6462fa6d8e0c1952b388824b,,dispatch,"{""first_mile_distance"":{""bucket"":{""params"":[{""...","[{""filter"":[{""meta"":{""first_mile_percent_slack...","[{""earnings"":{""amount"":30.41,""preCommissionAmo...",5bc5ac2312477c2ece769591,5d0f9806c2a56b449fa14d5c,"{""lat"":28.4511202,""lng"":77.0965147}",Delhi,"{""id"":""6462fa6d8e0c1952b388824b"",""type"":""order""}",1770.0,app,rapido,"{""lat"":28.453746648990034,""lng"":77.0866435021162}",app,5da4660028af187d8d52cc3c,0,"[{""captain_id"":""636f8c0780632c4e2e6f8639"",""cap...",None,None,None,None,20230516,09,0900
3,1684208526459,44c48d21-17e7-4217-b086-39c38a899376,1684208526459,1.3.0,"[""economicFeasibilityOfARide"",""failedMatchInTh...",6462fb8d9d1dd7505bb85d6e,,dispatch,"{""first_mile_distance"":{""bucket"":{""params"":[{""...",[],"[{""earnings"":{""amount"":77.12,""preCommissionAmo...",5bc5ac2312477c2ece769591,5e27c4bb140560c235f03470,"{""lat"":28.4226457,""lng"":77.0376039}",Delhi,"{""id"":""6462fb8d9d1dd7505bb85d6e"",""type"":""order""}",7660.0,app,qrcode,"{""lat"":28.45247262182255,""lng"":77.07737680524588}",app,5da4660028af187d8d52cc3c,0,"[{""captain_id"":""5bbc7ea84eac1c2fd102b9a1"",""cap...",None,None,None,None,20230516,09,0900
4,1684210452476,d6617278-46c5-169a-e461-bcfc4331964c,1684210452476,1.3.0,"[""economicFeasibilityOfARide"",""failedMatchInTh...",646303149d1dd7505bb86f37,,dispatch,"{""first_mile_distance"":{""bucket"":{""params"":[{""...","[{""filter"":[{""meta"":{""first_mile_percent_slack...","[{""earnings"":{""amount"":33.75,""preCommissionAmo...",5bc5ac2312477c2ece769591,5e67934248d7c7894c00ce3e,"{""lat"":28.4538136,""lng"":77.0973243}",Delhi,"{""id"":""646303149d1dd7505bb86f37"",""type"":""order""}",1750.0,app,qrcode,"{""lat"":28.4543812986275,""lng"":77.08618953824043}",app,5da4660028af187d8d52cc3c,0,"[{""captain_id"":""621624f9656bb35c899caf36"",""cap...",None,None,None,None,20230516,09,0930


In [147]:
df_dispatch2['captains_with_score'] = df_dispatch2['captains_with_score'].apply(lambda x: json.loads(x))
df_dispatch2.head()

,created_on,event_id,updated_at,version,enabled_captain_filter_types,order_id,compositeHexId,prioritization_stage,enabled_strategies_with_config,filtered_captains,captains,city_id,customer_id,drop_location,city_name,correlation,last_mile_distance,order_type,payment_type,pickup_location,service_category,service_detail_id,retry_count,captains_with_score,enabled_strategies_with_scores,epkm,infeasible_captains,prioritized_captains,yyyymmdd,hh,quarter_hour
0,1684209910717,92980715-8bcb-7b68-910d-6545a8b97ab5,1684209910717,1.3.0,"[""economicFeasibilityOfARide"",""failedMatchInTh...",646300f69ad8e2253f17829c,,dispatch,"{""first_mile_distance"":{""bucket"":{""params"":[{""...",[],"[{""earnings"":{""amount"":53.87,""preCommissionAmo...",5bc5ac2312477c2ece769591,5d1580563b752c45cf8bce8e,"{""lat"":28.459147,""lng"":77.072858}",Delhi,"{""id"":""646300f69ad8e2253f17829c"",""type"":""order""}",4770.0,app,cash,"{""lat"":28.45348109269419,""lng"":77.09132214882767}",app,5da4660028af187d8d52cc3c,0,"[{'captain_id': '644e0a66644efb1c5fee0bd9', 'c...",None,None,None,None,20230516,09,0930
1,1684210168813,49842c6a-ac9d-3863-1d4b-1665bc8be8ec,1684210168813,1.3.0,"[""economicFeasibilityOfARide"",""failedMatchInTh...",64630197a5c51b1fbd43d008,,dispatch,"{""first_mile_distance"":{""bucket"":{""params"":[{""...",[],"[{""earnings"":{""amount"":95.02,""preCommissionAmo...",5bc5ac2312477c2ece769591,62bcd17417af9a5cd065dda8,"{""lat"":28.412011773817405,""lng"":77.10839923471...",Delhi,"{""id"":""64630197a5c51b1fbd43d008"",""type"":""order""}",9400.0,app,cash,"{""lat"":28.455322801568695,""lng"":77.08740960806...",app,5da4660028af187d8d52cc3c,0,"[{'captain_id': '5fb55921e41162525b22df8f', 'c...",None,None,None,None,20230516,09,0930
2,1684208237670,c5c24b54-5d38-335c-46fc-d76c6e08e6ea,1684208237670,1.3.0,"[""economicFeasibilityOfARide"",""failedMatchInTh...",6462fa6d8e0c1952b388824b,,dispatch,"{""first_mile_distance"":{""bucket"":{""params"":[{""...","[{""filter"":[{""meta"":{""first_mile_percent_slack...","[{""earnings"":{""amount"":30.41,""preCommissionAmo...",5bc5ac2312477c2ece769591,5d0f9806c2a56b449fa14d5c,"{""lat"":28.4511202,""lng"":77.0965147}",Delhi,"{""id"":""6462fa6d8e0c1952b388824b"",""type"":""order""}",1770.0,app,rapido,"{""lat"":28.453746648990034,""lng"":77.0866435021162}",app,5da4660028af187d8d52cc3c,0,"[{'captain_id': '636f8c0780632c4e2e6f8639', 'c...",None,None,None,None,20230516,09,0900
3,1684208526459,44c48d21-17e7-4217-b086-39c38a899376,1684208526459,1.3.0,"[""economicFeasibilityOfARide"",""failedMatchInTh...",6462fb8d9d1dd7505bb85d6e,,dispatch,"{""first_mile_distance"":{""bucket"":{""params"":[{""...",[],"[{""earnings"":{""amount"":77.12,""preCommissionAmo...",5bc5ac2312477c2ece769591,5e27c4bb140560c235f03470,"{""lat"":28.4226457,""lng"":77.0376039}",Delhi,"{""id"":""6462fb8d9d1dd7505bb85d6e"",""type"":""order""}",7660.0,app,qrcode,"{""lat"":28.45247262182255,""lng"":77.07737680524588}",app,5da4660028af187d8d52cc3c,0,"[{'captain_id': '5bbc7ea84eac1c2fd102b9a1', 'c...",None,None,None,None,20230516,09,0900
4,1684210452476,d6617278-46c5-169a-e461-bcfc4331964c,1684210452476,1.3.0,"[""economicFeasibilityOfARide"",""failedMatchInTh...",646303149d1dd7505bb86f37,,dispatch,"{""first_mile_distance"":{""bucket"":{""params"":[{""...","[{""filter"":[{""meta"":{""first_mile_percent_slack...","[{""earnings"":{""amount"":33.75,""preCommissionAmo...",5bc5ac2312477c2ece769591,5e67934248d7c7894c00ce3e,"{""lat"":28.4538136,""lng"":77.0973243}",Delhi,"{""id"":""646303149d1dd7505bb86f37"",""type"":""order""}",1750.0,app,qrcode,"{""lat"":28.4543812986275,""lng"":77.08618953824043}",app,5da4660028af187d8d52cc3c,0,"[{'captain_id': '621624f9656bb35c899caf36', 'c...",None,None,None,None,20230516,09,0930


In [45]:
df_dispatch2['mc_captain'] = df_dispatch2['captains_with_score'].apply(lambda x:
                                                                     [captain['captain_id'] for captain in x if captain['captain_id'] == '5daff8e0b2d9ba1c1cf1bdf9'])

In [46]:
df_filter2 = df_dispatch2[df_dispatch2['mc_captain'].astype(str) != '[]']

In [47]:
orders = df_dispatch2['order_id'].unique().tolist()
print(orders)

['6461d5a1a78efb0c7dc71e6e', '6461d60c1e1f7e2e3765351d', '6461d9b297b0a30bbd66f587', '6461d59cf4a8ae312cd1e0f8', '6461dbcfa78efb0c7dc72b91', '6461d5bd5ac5d8625a819502', '6461df4bf4a8ae312cd1f650', '6461d5fecd5d212b73a0e034', '6461dbb91e1f7e2e37654165', '6461dc088eac592e5cae996a', '6461dbc09f166a55ff604ba6', '6461d62b612b71236703b103', '6461d754780955423c322f8d', '6461db90b97fe77d4ecf91b7', '6461de08cd5d212b73a0f1cc', '6461de1df4a8ae312cd1f39c', '6461de869c50b541b2df2f1c', '6461d7141e1f7e2e37653760', '6461d9dc1e1f7e2e37653d86', '6461d82f5ac5d8625a819a3a', '6461d94f780955423c3233cd', '6461dd89780955423c323d29', '6461d8cf8956b6528798490c', '6461dedf9c50b541b2df2fc2', '6461dee4b97fe77d4ecf98c6', '6461df76751ce46ce048fbb9', '6461dee44358375373989aed', '6461dc9b612b71236703bf5b', '6461df5597b0a30bbd6701d5', '6461ddcc8eac592e5cae9d4e', '6461dd0c9f166a55ff604e47', '6461dde51e1f7e2e3765465f', '6461da69ca5873133f31585d', '6461dae5ca5873133f31596f', '6461dce9ad68a90a314dce9f', '6461dd46cd5d212b73

In [48]:
orders_not_in_suppri_logs2 = []
for order in second_captain_orders:
    if order not in orders:
        orders_not_in_suppri_logs2.append(order)

In [49]:
print(orders_not_in_suppri_logs2)

['6461dccd8753985ab62d977e', '6461dbeb8753985ab62d9584', '6461d74e751ce46ce048ea48', '6461d9938753985ab62d9080', '6461dae5cd5d212b73a0eb1e', '6461da5c9c50b541b2df2630', '6461dcf643583753739896d8', '6461d823780955423c32315d', '6461da858eac592e5cae964c', '6461d72d8753985ab62d8b4b', '6461d6ff43583753739889d9', '6461d64db97fe77d4ecf861b', '6461d6b61e1f7e2e3765368a', '6461dd2fa78efb0c7dc72e9a', '6461d8d39c50b541b2df22f7', '6461dcaf8eac592e5cae9ab3', '6461db618eac592e5cae9809', '6461de36751ce46ce048f8fe', '6461d79eb97fe77d4ecf8910', '6461de5a5ac5d8625a81a759', '6461d6f643583753739889c8', '6461dc8943583753739895fc', '6461d75d8753985ab62d8bb2', '6461d7ff612b71236703b50f', '6461d9b9780955423c3234d6', '6461dda697b0a30bbd66fe36', '6461d7c08956b652879846c7']


In [151]:
fm_distance2 = []
captain_rank2 = []
micro_corridor2 = []
captain_list2 = df_dispatch2['captains_with_score'].values
for captains2 in captain_list2:
    for captain2 in captains2:
        if captain2['captain_id'] == '59b153ef0c3e1ca306385c51':
            fm_distance2.append(captain2['first_mile_distance'])
            captain_rank2.append(captains2.index(captain2))
            micro_corridor2.append(captain2['score']['micro_corridor'])

In [214]:
captain_list2 = df_dispatch2['captains_with_score'].values
for captains2 in captain_list2:
    for captain2 in captains2:
        print(captain2['first_mile_distance'])

869.3
1004.9
1756.2
1798.8
1807.3
2026.3
2047.4
2313.5
2322.8
2405.3
2526.6
2694.1
2780.0
2859.2
3030.3
3085.5
3146.0
3156.6
3451.0
3497.1
3535.9
3591.6
3620.9
3632.8
3828.2
4425.2
4756.9
662.6
1852.2
2258.2
2315.2
2477.5
2505.6
2660.7
2946.3
2963.3
2976.1
3036.7
3200.5
3327.8
3801.8
3873.1
5409.8
328.1
682.3
721.2
1347.1
1418.7
1507.0
1813.3
1896.4
2088.5
2138.3
2147.6
2182.1
2241.2
2282.2
2368.3
2387.5
2388.1
2390.9
2410.2
2620.0
2646.8
2658.7
2779.8
2802.8
2805.3
2808.9
2819.1
2822.0
2874.0
2906.6
2912.9
2951.7
2981.9
2985.4
2997.2
3004.1
3023.4
3038.9
3080.4
3082.4
3162.0
3163.0
3180.7
3201.6
3233.8
3237.9
3260.6
3273.5
3305.9
3307.5
3315.9
3320.0
3337.8
3343.6
3381.2
3492.3
870.4
1000.7
1004.6
1058.8
1189.5
1250.6
1273.9
1282.7
1285.0
1287.6
1299.7
1308.3
1364.5
1434.2
1441.2
1454.9
1620.2
1637.6
1651.9
1689.8
1692.7
1761.6
1768.9
1778.3
1780.8
1808.9
1842.4
1847.7
1855.2
1899.9
1926.5
1939.9
1954.8
2010.9
2047.7
2055.2
2076.2
2214.5
2221.2
2266.4
2323.0
2540.6
2558.3
2596.0
2807.

In [ ]:
3404/(3404+30540)

0.1002828187603111

In [152]:
print(fm_distance2)

[2526.6, 2315.2, 2241.2, 2055.2, 2244.5, 2353.2, 2276.1, 3707.1, 2369.0, 2361.5, 2633.6, 2775.7, 2316.2, 2574.7, 2154.0, 2298.5, 2257.9, 2225.1, 2182.1, 2772.5, 2772.5, 2377.5, 3025.0, 3614.8, 2622.9, 2249.8, 2361.0, 2786.9, 2366.3, 2263.3, 2312.4, 2650.9, 2679.8, 2269.3, 3750.5, 2721.7, 2117.7, 2563.0, 2696.2, 2380.8, 2154.0, 2448.3, 2990.9, 2744.8, 2112.0, 2731.1, 2311.6, 2151.6, 2194.9, 3601.3, 2167.4, 2917.1, 2855.9, 2374.0, 2836.2, 2361.6, 2667.8, 2155.1, 3591.5, 2245.7, 2225.5, 2671.3, 2447.0, 2288.6, 2814.4, 2378.5, 2314.5, 2304.5, 3325.5, 2218.9, 2864.2, 2330.4, 3829.1, 2301.7, 2163.0, 3379.7, 2216.3, 2185.3, 2269.4, 3595.7, 3591.0, 2607.7, 3035.6, 2999.9, 2230.0, 2909.0, 2723.2, 2270.4, 2208.1, 2632.4, 2355.8, 2128.3, 2112.9, 2357.1, 2154.5, 2446.5, 2294.7, 3816.7, 2796.6, 2795.3, -1.0, 2528.7, 3519.0, 2475.8, 2348.7, 2449.7, 2418.4, 2469.0, 2273.0, 2423.3, 2293.0, 2748.2, 2548.7, 2224.1, 2449.3, 2201.8, 3780.7, 3110.4, 2359.2, 2868.4, 2303.9, 3780.6, 2380.3, 2194.3, 3412.2, 2

In [153]:
print(micro_corridor2)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [65]:
#Query data from order_logs_snapshot

q = """
select order_id, accept_to_pickup_distance, city_name, customer_location_hex_8, customer_location_latitude, 
customer_location_longitude, hhmmss, hour, quarter_hour, service_obj_service_name, time_bucket, weekday, yyyymmdd,
order_status, cancel_reason, eta, map_riders, pickup_location_hex_8, captain_id, event_type
from orders.order_logs_immutable
where yyyymmdd = '20230515'
and city_name in ('Delhi')
and service_obj_service_name = 'Link'
and captain_id in ('5daff8e0b2d9ba1c1cf1bdf9', '59b153ef0c3e1ca306385c51')
and event_type in ('accepted', 'rider_busy', 'rider_reject')
"""


#Load data into pandas table
df_apr_copy = pd.read_sql(q, connection)

In [74]:
print(len(df_apr_copy['order_id'].unique().tolist()))

93


In [67]:
#Query data from order_logs_snapshot

q = """
select * from g2n.log_dispatch_prioritiser
where yyyymmdd = '20230515'
and service_detail_id = '5da4660028af187d8d52cc3c'
and order_id in ('6461e97297b0a30bbd6717f2', '6461e8deca5873133f31786a', '6461ea1ea78efb0c7dc74aba', '6461df7c8eac592e5caea0fd', '6461e7921e1f7e2e37655b91',
 '6461e89d97b0a30bbd671622', '6461e99ef4a8ae312cd20da0', '6461e451ca5873133f316e3d', '6461ea98ca5873133f317c3f', '6461eafb612b71236703ded8',
   '6461e8101e1f7e2e37655c98', '6461df998eac592e5caea132', '6461e03c780955423c324321', '6461e72eb97fe77d4ecfab27', '6461eb705ac5d8625a81c47e',
     '6461e4f6a78efb0c7dc73f69', '64621408780955423c32acd0', '6461ee2b612b71236703e5c3', '6461e3ea9f166a55ff605cee', '6461eac29c50b541b2df4a40',
       '6461e84197b0a30bbd671561', '6461eaa7612b71236703de20', '6461e851cd5d212b73a10893', '6461e7239c50b541b2df4203', '6461e22f9c50b541b2df373b',
         '6461e171435837537398a07f', '6462138e780955423c32abc8', '6461e9158956b65287986d39', '6461e9df1e1f7e2e3765608a', '6461ece01e1f7e2e3765673f',
           '6461e55c751ce46ce04908ef', '6461ed5c97b0a30bbd672075', '6461e3429f166a55ff605b7c', '6461e5dc612b71236703d397', '6461ec62cd5d212b73a11110',
             '6461e6c58eac592e5caeb15b', '6461e9f48eac592e5caeb85c', '6461e67b1e1f7e2e37655913', '6461edeef4a8ae312cd216da', '6461e912b97fe77d4ecfaf2c',
               '6461e26f435837537398a2e1', '6461e480ca5873133f316ec6', '6462116fca5873133f31c939', '6461e357ca5873133f316bf7', '6461ef6ea78efb0c7dc7562d',
                 '6461ebca9f166a55ff606e0a', '6461e87697b0a30bbd6715ca', '6461f067ad68a90a314df871', '6461d9b9780955423c3234d6', '6461dc8f780955423c323b03',
                   '6461e8235ac5d8625a81bd4e', '6461dbeeca5873133f315bb3', '6461d9938753985ab62d9080', '6461e52d612b71236703d224', '6461db618eac592e5cae9809',
                     '6461d8d39c50b541b2df22f7', '6461da858eac592e5cae964c', '6461de36751ce46ce048f8fe', '6461d8fe8eac592e5cae933e', '6461da31b97fe77d4ecf8eac',
                       '6461dcf643583753739896d8', '6461e1ca8eac592e5caea606', '6461d760780955423c322faa', '6461dc1697b0a30bbd66fac2', '6461dcc0780955423c323b6f',
                         '6461d79eb97fe77d4ecf8910', '6461dc8943583753739895fc', '6461df4fcd5d212b73a0f49d', '6461d6b61e1f7e2e3765368a', '6461decbf4a8ae312cd1f51e',
                           '6461dcdcad68a90a314dce87', '6461d72d8753985ab62d8b4b', '6461d64db97fe77d4ecf861b', '6461dd2fa78efb0c7dc72e9a', '6461d823780955423c32315d',
                             '6461dd2cca5873133f315e73', '6461d82f5ac5d8625a819a3a', '6461dc4f612b71236703beb4', '6461dae5cd5d212b73a0eb1e',
                               '6461d9385ac5d8625a819c48', '6461dda697b0a30bbd66fe36', '6461d75d8753985ab62d8bb2', '6461db9343583753739893df',
                                 '6461d6ff43583753739889d9', '6461de5a5ac5d8625a81a759', '646246ce3c976113e233feb0', '6461de3f435837537398999e',
                                   '6461dbeb8753985ab62d9584', '6461dccd8753985ab62d977e', '6461ddfccd5d212b73a0f1b1', '6461deed751ce46ce048fa74',
                                     '6461d969b97fe77d4ecf8d01', '6461df2c8753985ab62d9c8f')
"""


#Load data into pandas table
df_dispatch_copy2 = pd.read_sql(q, connection)

In [68]:
df_dispatch_copy2

,created_on,event_id,updated_at,version,enabled_captain_filter_types,order_id,compositeHexId,prioritization_stage,enabled_strategies_with_config,filtered_captains,captains,city_id,customer_id,drop_location,city_name,correlation,last_mile_distance,order_type,payment_type,pickup_location,service_category,service_detail_id,retry_count,captains_with_score,enabled_strategies_with_scores,epkm,infeasible_captains,prioritized_captains,yyyymmdd,hh,quarter_hour
0,1684140292418,7135eb5c-0a29-e9de-26b8-be863d5ffb81,1684140292418,1.3.0,"[""economicFeasibilityOfARide"",""failedMatchInTh...",6461f067ad68a90a314df871,,dispatch,"{""first_mile_distance"":{""bucket"":{""params"":[{""...","[{""filter"":[{""meta"":{""first_mile_percent_slack...","[{""earnings"":{""amount"":31.93,""preCommissionAmo...",5bc5ac2312477c2ece769591,5d8e0c04a1f0a71c604b69f5,"{""lat"":28.4527467,""lng"":77.0510669}",Delhi,"{""id"":""6461f067ad68a90a314df871"",""type"":""order""}",1090.0,app,cash,"{""lat"":28.4542353855553,""lng"":77.05701012164354}",app,5da4660028af187d8d52cc3c,0,"[{""captain_id"":""63b2d840cb0cad3e7cbac392"",""cap...",None,None,None,None,20230515,14,1400
1,1684139563474,05253f4d-8209-f8e5-3fd8-b2953abf828e,1684139563474,1.3.0,"[""economicFeasibilityOfARide"",""failedMatchInTh...",6461ee2b612b71236703e5c3,,dispatch,"{""first_mile_distance"":{""bucket"":{""params"":[{""...",[],"[{""earnings"":{""amount"":37.01,""preCommissionAmo...",5bc5ac2312477c2ece769591,618c9773c98cb31c05fb2266,"{""lat"":28.474934,""lng"":77.084606}",Delhi,"{""id"":""6461ee2b612b71236703e5c3"",""type"":""order""}",3250.0,app,paytm,"{""lat"":28.454235975103476,""lng"":77.08387613296...",app,5da4660028af187d8d52cc3c,0,"[{""captain_id"":""5daff8e0b2d9ba1c1cf1bdf9"",""cap...",None,None,None,None,20230515,14,1400
2,1684135834295,d0311a16-ef5c-4f66-c522-52cd284ac53b,1684135834295,1.3.0,"[""economicFeasibilityOfARide"",""failedMatchInTh...",6461df998eac592e5caea132,,dispatch,"{""first_mile_distance"":{""bucket"":{""params"":[{""...","[{""filter"":[{""meta"":{""first_mile_percent_slack...","[{""earnings"":{""amount"":27.57,""preCommissionAmo...",5bc5ac2312477c2ece769591,637b581ffbd64e460e8f14f1,"{""lat"":28.4524237,""lng"":77.069357}",Delhi,"{""id"":""6461df998eac592e5caea132"",""type"":""order""}",2720.0,app,paytm,"{""lat"":28.46185325257259,""lng"":77.08687350153923}",app,5da4660028af187d8d52cc3c,0,"[{""captain_id"":""5d10ac1a0eed7c44be2c1fb5"",""cap...",None,None,None,None,20230515,13,1300
3,1684139945562,5bd9cdb5-8473-671e-2b04-9f2c3b933453,1684139945562,1.3.0,"[""economicFeasibilityOfARide"",""failedMatchInTh...",6461ef6ea78efb0c7dc7562d,,dispatch,"{""first_mile_distance"":{""bucket"":{""params"":[{""...","[{""filter"":[{""meta"":{""first_mile_percent_slack...","[{""earnings"":{""amount"":42.61,""preCommissionAmo...",5bc5ac2312477c2ece769591,640326b98e212c0c234dd93a,"{""lat"":28.4542434,""lng"":77.0851923}",Delhi,"{""id"":""6461ef6ea78efb0c7dc7562d"",""type"":""order""}",2810.0,app,cash,"{""lat"":28.458548709966106,""lng"":77.07276653274...",app,5da4660028af187d8d52cc3c,0,"[{""captain_id"":""635cc2bed58af76104dc3473"",""cap...",None,None,None,None,20230515,14,1400
4,1684140136342,20f41813-4c08-6af6-e658-6b0781c35555,1684140136342,1.3.0,"[""economicFeasibilityOfARide"",""failedMatchInTh...",6461f067ad68a90a314df871,,dispatch,"{""first_mile_distance"":{""bucket"":{""params"":[{""...","[{""filter"":[{""meta"":{""first_mile_percent_slack...","[{""earnings"":{""amount"":24.76,""preCommissionAmo...",5bc5ac2312477c2ece769591,5d8e0c04a1f0a71c604b69f5,"{""lat"":28.4527467,""lng"":77.0510669}",Delhi,"{""id"":""6461f067ad68a90a314df871"",""type"":""order""}",1090.0,app,cash,"{""lat"":28.4542353855553,""lng"":77.05701012164354}",app,5da4660028af187d8d52cc3c,0,"[{""captain_id"":""63aa660aff12f9b46e7e6c6f"",""cap...",None,None,None,None,20230515,14,1400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174,1684136939711,65d3f

In [75]:
df_dispatch_copy2['order_id'].nunique()

59

In [78]:
df_dispatch_copy2['captains_with_score'] = df_dispatch_copy2['captains_with_score'].apply(lambda x: json.loads(x))
df_dispatch_copy2.head()

,created_on,event_id,updated_at,version,enabled_captain_filter_types,order_id,compositeHexId,prioritization_stage,enabled_strategies_with_config,filtered_captains,captains,city_id,customer_id,drop_location,city_name,correlation,last_mile_distance,order_type,payment_type,pickup_location,service_category,service_detail_id,retry_count,captains_with_score,enabled_strategies_with_scores,epkm,infeasible_captains,prioritized_captains,yyyymmdd,hh,quarter_hour
0,1684140292418,7135eb5c-0a29-e9de-26b8-be863d5ffb81,1684140292418,1.3.0,"[""economicFeasibilityOfARide"",""failedMatchInTh...",6461f067ad68a90a314df871,,dispatch,"{""first_mile_distance"":{""bucket"":{""params"":[{""...","[{""filter"":[{""meta"":{""first_mile_percent_slack...","[{""earnings"":{""amount"":31.93,""preCommissionAmo...",5bc5ac2312477c2ece769591,5d8e0c04a1f0a71c604b69f5,"{""lat"":28.4527467,""lng"":77.0510669}",Delhi,"{""id"":""6461f067ad68a90a314df871"",""type"":""order""}",1090.0,app,cash,"{""lat"":28.4542353855553,""lng"":77.05701012164354}",app,5da4660028af187d8d52cc3c,0,"[{'captain_id': '63b2d840cb0cad3e7cbac392', 'c...",None,None,None,None,20230515,14,1400
1,1684139563474,05253f4d-8209-f8e5-3fd8-b2953abf828e,1684139563474,1.3.0,"[""economicFeasibilityOfARide"",""failedMatchInTh...",6461ee2b612b71236703e5c3,,dispatch,"{""first_mile_distance"":{""bucket"":{""params"":[{""...",[],"[{""earnings"":{""amount"":37.01,""preCommissionAmo...",5bc5ac2312477c2ece769591,618c9773c98cb31c05fb2266,"{""lat"":28.474934,""lng"":77.084606}",Delhi,"{""id"":""6461ee2b612b71236703e5c3"",""type"":""order""}",3250.0,app,paytm,"{""lat"":28.454235975103476,""lng"":77.08387613296...",app,5da4660028af187d8d52cc3c,0,"[{'captain_id': '5daff8e0b2d9ba1c1cf1bdf9', 'c...",None,None,None,None,20230515,14,1400
2,1684135834295,d0311a16-ef5c-4f66-c522-52cd284ac53b,1684135834295,1.3.0,"[""economicFeasibilityOfARide"",""failedMatchInTh...",6461df998eac592e5caea132,,dispatch,"{""first_mile_distance"":{""bucket"":{""params"":[{""...","[{""filter"":[{""meta"":{""first_mile_percent_slack...","[{""earnings"":{""amount"":27.57,""preCommissionAmo...",5bc5ac2312477c2ece769591,637b581ffbd64e460e8f14f1,"{""lat"":28.4524237,""lng"":77.069357}",Delhi,"{""id"":""6461df998eac592e5caea132"",""type"":""order""}",2720.0,app,paytm,"{""lat"":28.46185325257259,""lng"":77.08687350153923}",app,5da4660028af187d8d52cc3c,0,"[{'captain_id': '5d10ac1a0eed7c44be2c1fb5', 'c...",None,None,None,None,20230515,13,1300
3,1684139945562,5bd9cdb5-8473-671e-2b04-9f2c3b933453,1684139945562,1.3.0,"[""economicFeasibilityOfARide"",""failedMatchInTh...",6461ef6ea78efb0c7dc7562d,,dispatch,"{""first_mile_distance"":{""bucket"":{""params"":[{""...","[{""filter"":[{""meta"":{""first_mile_percent_slack...","[{""earnings"":{""amount"":42.61,""preCommissionAmo...",5bc5ac2312477c2ece769591,640326b98e212c0c234dd93a,"{""lat"":28.4542434,""lng"":77.0851923}",Delhi,"{""id"":""6461ef6ea78efb0c7dc7562d"",""type"":""order""}",2810.0,app,cash,"{""lat"":28.458548709966106,""lng"":77.07276653274...",app,5da4660028af187d8d52cc3c,0,"[{'captain_id': '635cc2bed58af76104dc3473', 'c...",None,None,None,None,20230515,14,1400
4,1684140136342,20f41813-4c08-6af6-e658-6b0781c35555,1684140136342,1.3.0,"[""economicFeasibilityOfARide"",""failedMatchInTh...",6461f067ad68a90a314df871,,dispatch,"{""first_mile_distance"":{""bucket"":{""params"":[{""...","[{""filter"":[{""meta"":{""first_mile_percent_slack...","[{""earnings"":{""amount"":24.76,""preCommissionAmo...",5bc5ac2312477c2ece769591,5d8e0c04a1f0a71c604b69f5,"{""lat"":28.4527467,""lng"":77.0510669}",Delhi,"{""id"":""6461f067ad68a90a314df871"",""type"":""order""}",1090.0,app,cash,"{""lat"":28.4542353855553,""lng"":77.05701012164354}",app,5da4660028af187d8d52cc3c,0,"[{'captain_id': '63aa660aff12f9b46e7e6c6f', 'c...",None,None,None,None,20230515,14,1400


In [82]:
fm_distance2 = []
captain_rank2 = []
micro_corridor2 = []
fm_distance = []
captain_rank = []
micro_corridor = []
captain_list2 = df_dispatch_copy2['captains_with_score'].values
for captains2 in captain_list2:
    for captain2 in captains2:
        if captain2['captain_id'] == '5daff8e0b2d9ba1c1cf1bdf9':
            fm_distance2.append(captain2['first_mile_distance'])
            captain_rank2.append(captains2.index(captain2))
            micro_corridor2.append(captain2['micro_corridor_enabled'])
        elif captain2['captain_id'] == '59b153ef0c3e1ca306385c51':
            fm_distance.append(captain2['first_mile_distance'])
            captain_rank.append(captains2.index(captain2))
            micro_corridor.append(captain2['micro_corridor_enabled'])

In [187]:
for x in df_dispatch_copy2['captains_with_score'].values[0]:
    if x['captain_id'] == '59b153ef0c3e1ca306385c51':
        print(x['location'])

{'lat': 28.438816692504904, 'lng': 77.07940354943275}


In [189]:
h3.geo_to_h3(28.438816692504904, 77.07940354943275, resolution=8)

'883da11ab5fffff'

In [83]:
print(fm_distance2)

[407.9, 1117.0, 1112.4, 498.9, 347.0, 298.7, 3443.0, 396.3, 1777.6, 2344.0, 1795.7, 3429.6, 345.2, 328.9, 1136.3, 3388.0, 3737.8, 47.4, 735.0, 3519.1, 2793.8, 2793.0, 1161.0, 1704.4, 3443.1, 1796.4, 3395.4, 3737.3, 3750.8, 3441.7, 1152.3, 1819.5, 3395.4, 730.8, 88.2, 2366.0, 1133.5, 2793.9, 3381.0, 1104.6, 914.2, 1938.3, 3518.2, 1953.2, 1792.9, 27.2, 1191.7, 1787.5, 3434.9, 2244.0, 340.8, 2194.8, 1090.6, 1203.6, 1777.6, 90.2, 2369.2, 2744.0, 1793.4, 920.4, 2391.1, 1936.3, 2127.0, 1679.7, 1931.3, 1775.8, 178.8, 102.6, 25.7, 2544.4, 2786.8, 1159.7, 2195.1, 44.5, 3531.0, 2585.0, 2550.8, 1580.9, 1693.8, 3743.4, 2247.2, 2751.4, 1795.9]


In [84]:
print(fm_distance)

[938.4, 938.4, 2290.0, 684.1, 496.1, 935.8, 337.1, 24.0, 2463.7]


In [85]:
print(captain_rank)

[4, 3, 1, 1, 1, 0, 2, 0, 52]


In [87]:
df_dispatch_copy2['mc_captain_1'] = df_dispatch_copy2['captains_with_score'].apply(lambda x:
                                                                     [captain['captain_id'] for captain in x if captain['captain_id'] == '5daff8e0b2d9ba1c1cf1bdf9'])
df_dispatch_copy2['mc_captain_2'] = df_dispatch_copy2['captains_with_score'].apply(lambda x:
                                                                     [captain['captain_id'] for captain in x if captain['captain_id'] == '59b153ef0c3e1ca306385c51'])

In [120]:
df_filter2 = df_dispatch_copy2[df_dispatch_copy2['mc_captain_2'].astype(str) != '[]']
df_filter2

,created_on,event_id,updated_at,version,enabled_captain_filter_types,order_id,compositeHexId,prioritization_stage,enabled_strategies_with_config,filtered_captains,captains,city_id,customer_id,drop_location,city_name,correlation,last_mile_distance,order_type,payment_type,pickup_location,service_category,service_detail_id,retry_count,captains_with_score,enabled_strategies_with_scores,epkm,infeasible_captains,prioritized_captains,yyyymmdd,hh,quarter_hour,mc_captain_1,mc_captain_2
4,1684140136342,20f41813-4c08-6af6-e658-6b0781c35555,1684140136342,1.3.0,"[""economicFeasibilityOfARide"",""failedMatchInTh...",6461f067ad68a90a314df871,,dispatch,"{""first_mile_distance"":{""bucket"":{""params"":[{""...","[{""filter"":[{""meta"":{""first_mile_percent_slack...","[{""earnings"":{""amount"":24.76,""preCommissionAmo...",5bc5ac2312477c2ece769591,5d8e0c04a1f0a71c604b69f5,"{""lat"":28.4527467,""lng"":77.0510669}",Delhi,"{""id"":""6461f067ad68a90a314df871"",""type"":""order""}",1090.0,app,cash,"{""lat"":28.4542353855553,""lng"":77.05701012164354}",app,5da4660028af187d8d52cc3c,0,"[{'captain_id': '63aa660aff12f9b46e7e6c6f', 'c...",None,None,None,None,20230515,14,1400,[],[59b153ef0c3e1ca306385c51]
5,1684140171298,de01dc8a-ba2f-d9f3-bb80-8291681fde86,1684140171298,1.3.0,"[""economicFeasibilityOfARide"",""failedMatchInTh...",6461f067ad68a90a314df871,,dispatch,"{""first_mile_distance"":{""bucket"":{""params"":[{""...","[{""filter"":[{""meta"":{""first_mile_percent_slack...","[{""earnings"":{""amount"":24.76,""preCommissionAmo...",5bc5ac2312477c2ece769591,5d8e0c04a1f0a71c604b69f5,"{""lat"":28.4527467,""lng"":77.0510669}",Delhi,"{""id"":""6461f067ad68a90a314df871"",""type"":""order""}",1090.0,app,cash,"{""lat"":28.4542353855553,""lng"":77.05701012164354}",app,5da4660028af187d8d52cc3c,0,"[{'captain_id': '62863d9e8f9b4309675e2665', 'c...",None,None,None,None,20230515,14,1400,[],[59b153ef0c3e1ca306385c51]
14,1684140068293,026fe352-e279-6348-1d6f-c26c06ed1a48,1684140068293,1.3.0,"[""economicFeasibilityOfARide"",""failedMatchInTh...",6461ef6ea78efb0c7dc7562d,,dispatch,"{""first_mile_distance"":{""bucket"":{""params"":[{""...",[],"[{""firstMileDistance"":3318.7,""id"":""612085be01c...",5bc5ac2312477c2ece769591,640326b98e212c0c234dd93a,"{""lat"":28.4542434,""lng"":77.0851923}",Delhi,"{""id"":""6461ef6ea78efb0c7dc7562d"",""type"":""order""}",2810.0,app,cash,"{""lat"":28.458548709966106,""lng"":77.07276653274...",app,5da4660028af187d8d52cc3c,0,"[{'captain_id': '63b2d840cb0cad3e7cbac392', 'c...",None,None,None,None,20230515,14,1400,[],[59b153ef0c3e1ca306385c51]
18,1684149134719,b1018bd7-e22e-421f-a9eb-9f61d5cb2e9e,1684149134719,1.3.0,"[""economicFeasibilityOfARide"",""failedMatchInTh...",6462138e780955423c32abc8,,dispatch,"{""first_mile_distance"":{""bucket"":{""params"":[{""...","[{""filter"":[{""meta"":{""first_mile_percent_slack...","[{""earnings"":{""amount"":47.16,""preCommissionAmo...",5bc5ac2312477c2ece769591,600f2a5a027f256eb8a33e1e,"{""lat"":28.4312655684068,""lng"":77.0443594828248}",Delhi,"{""id"":""6462138e780955423c32abc8"",""type"":""order""}",4330.0,app,simpl,"{""lat"":28.459418566032934,""lng"":77.04956263303...",app,5da4660028af187d8d52cc3c,0,"[{'captain_id': '5de8ad2523da8b6b4b802cf5', 'c...",None,None,None,None,20230515,16,1630,[],[59b153ef0c3e1ca306385c51]
19,1684162255544,b298d76b-071f-a2c1-1053-34c6b0ac0ba8,1684162255544,1.3.0,"[""economicFeasibilityOfARide"",""failedMatchInTh...",646246ce3c976113e233feb0,,dispatch,"{""dropped_to_total_pings_ratio"":{""bucket"":{""pa...","[{""filter"":[{""meta"":{""first_mile_percent_slack...","[{""earnings"":{""amount"":20.0,""preCommissionAmou...",5bc5ac2312477c2ece769591,61c811d45379c0d4d24f1e92,"{""lat"":28.4392927,""lng"":77.0797819}",Delhi,"{""id"":""646246ce3c976113e233feb0"",""type"":""order""}",540.0,app,paytm,"{""lat"":28.4411481,""lng"":77.07658931}",app,5da4660028af187d8d52cc3c,0,"[{'captain_id': '644f68a2e80c7dfa1ead1ce9', 'c...",None,None,None,None,20230515,20,2015,[],[59b153ef0c3e1ca306385c51

In [111]:
df_filter2['enabled_strategies_with_config'].values[0]

'{"first_mile_distance":{"bucket":{"params":[{"primaryBucketEnd":1000,"primaryBucketStart":0,"secondaryBucketEnd":100000,"secondaryBucketStart":0,"weightage":100},{"primaryBucketEnd":2000,"primaryBucketStart":1000,"secondaryBucketEnd":100000,"secondaryBucketStart":0,"weightage":100},{"primaryBucketEnd":10000,"primaryBucketStart":2000,"secondaryBucketEnd":100000,"secondaryBucketStart":0,"weightage":100}],"type":"first_mile_distance"}}}'

In [92]:
fm_distance2 = []
captain_rank2 = []
micro_corridor2 = []
captain_list2 = df_filter2['captains_with_score'].values
for captains2 in captain_list2:
    for captain2 in captains2:
        if captain2['captain_id'] == '59b153ef0c3e1ca306385c51':
            fm_distance2.append(captain2['first_mile_distance'])
            captain_rank2.append(captains2.index(captain2))
            micro_corridor2.append(captain2['micro_corridor_enabled'])

In [93]:
print(captain_rank)

[4, 3, 1, 1, 1, 0, 2, 0, 52]


In [94]:
print(fm_distance)

[938.4, 938.4, 2290.0, 684.1, 496.1, 935.8, 337.1, 24.0, 2463.7]


In [121]:
df_filter2

,created_on,event_id,updated_at,version,enabled_captain_filter_types,order_id,compositeHexId,prioritization_stage,enabled_strategies_with_config,filtered_captains,captains,city_id,customer_id,drop_location,city_name,correlation,last_mile_distance,order_type,payment_type,pickup_location,service_category,service_detail_id,retry_count,captains_with_score,enabled_strategies_with_scores,epkm,infeasible_captains,prioritized_captains,yyyymmdd,hh,quarter_hour,mc_captain_1,mc_captain_2
4,1684140136342,20f41813-4c08-6af6-e658-6b0781c35555,1684140136342,1.3.0,"[""economicFeasibilityOfARide"",""failedMatchInTh...",6461f067ad68a90a314df871,,dispatch,"{""first_mile_distance"":{""bucket"":{""params"":[{""...","[{""filter"":[{""meta"":{""first_mile_percent_slack...","[{""earnings"":{""amount"":24.76,""preCommissionAmo...",5bc5ac2312477c2ece769591,5d8e0c04a1f0a71c604b69f5,"{""lat"":28.4527467,""lng"":77.0510669}",Delhi,"{""id"":""6461f067ad68a90a314df871"",""type"":""order""}",1090.0,app,cash,"{""lat"":28.4542353855553,""lng"":77.05701012164354}",app,5da4660028af187d8d52cc3c,0,"[{'captain_id': '63aa660aff12f9b46e7e6c6f', 'c...",None,None,None,None,20230515,14,1400,[],[59b153ef0c3e1ca306385c51]
5,1684140171298,de01dc8a-ba2f-d9f3-bb80-8291681fde86,1684140171298,1.3.0,"[""economicFeasibilityOfARide"",""failedMatchInTh...",6461f067ad68a90a314df871,,dispatch,"{""first_mile_distance"":{""bucket"":{""params"":[{""...","[{""filter"":[{""meta"":{""first_mile_percent_slack...","[{""earnings"":{""amount"":24.76,""preCommissionAmo...",5bc5ac2312477c2ece769591,5d8e0c04a1f0a71c604b69f5,"{""lat"":28.4527467,""lng"":77.0510669}",Delhi,"{""id"":""6461f067ad68a90a314df871"",""type"":""order""}",1090.0,app,cash,"{""lat"":28.4542353855553,""lng"":77.05701012164354}",app,5da4660028af187d8d52cc3c,0,"[{'captain_id': '62863d9e8f9b4309675e2665', 'c...",None,None,None,None,20230515,14,1400,[],[59b153ef0c3e1ca306385c51]
14,1684140068293,026fe352-e279-6348-1d6f-c26c06ed1a48,1684140068293,1.3.0,"[""economicFeasibilityOfARide"",""failedMatchInTh...",6461ef6ea78efb0c7dc7562d,,dispatch,"{""first_mile_distance"":{""bucket"":{""params"":[{""...",[],"[{""firstMileDistance"":3318.7,""id"":""612085be01c...",5bc5ac2312477c2ece769591,640326b98e212c0c234dd93a,"{""lat"":28.4542434,""lng"":77.0851923}",Delhi,"{""id"":""6461ef6ea78efb0c7dc7562d"",""type"":""order""}",2810.0,app,cash,"{""lat"":28.458548709966106,""lng"":77.07276653274...",app,5da4660028af187d8d52cc3c,0,"[{'captain_id': '63b2d840cb0cad3e7cbac392', 'c...",None,None,None,None,20230515,14,1400,[],[59b153ef0c3e1ca306385c51]
18,1684149134719,b1018bd7-e22e-421f-a9eb-9f61d5cb2e9e,1684149134719,1.3.0,"[""economicFeasibilityOfARide"",""failedMatchInTh...",6462138e780955423c32abc8,,dispatch,"{""first_mile_distance"":{""bucket"":{""params"":[{""...","[{""filter"":[{""meta"":{""first_mile_percent_slack...","[{""earnings"":{""amount"":47.16,""preCommissionAmo...",5bc5ac2312477c2ece769591,600f2a5a027f256eb8a33e1e,"{""lat"":28.4312655684068,""lng"":77.0443594828248}",Delhi,"{""id"":""6462138e780955423c32abc8"",""type"":""order""}",4330.0,app,simpl,"{""lat"":28.459418566032934,""lng"":77.04956263303...",app,5da4660028af187d8d52cc3c,0,"[{'captain_id': '5de8ad2523da8b6b4b802cf5', 'c...",None,None,None,None,20230515,16,1630,[],[59b153ef0c3e1ca306385c51]
19,1684162255544,b298d76b-071f-a2c1-1053-34c6b0ac0ba8,1684162255544,1.3.0,"[""economicFeasibilityOfARide"",""failedMatchInTh...",646246ce3c976113e233feb0,,dispatch,"{""dropped_to_total_pings_ratio"":{""bucket"":{""pa...","[{""filter"":[{""meta"":{""first_mile_percent_slack...","[{""earnings"":{""amount"":20.0,""preCommissionAmou...",5bc5ac2312477c2ece769591,61c811d45379c0d4d24f1e92,"{""lat"":28.4392927,""lng"":77.0797819}",Delhi,"{""id"":""646246ce3c976113e233feb0"",""type"":""order""}",540.0,app,paytm,"{""lat"":28.4411481,""lng"":77.07658931}",app,5da4660028af187d8d52cc3c,0,"[{'captain_id': '644f68a2e80c7dfa1ead1ce9', 'c...",None,None,None,None,20230515,20,2015,[],[59b153ef0c3e1ca306385c51

In [114]:
captains_logs = df_filter2['captains_with_score'].values
for log in captains_logs:
    for individual_log in log:
        if individual_log['captain_id'] == '5daff8e0b2d9ba1c1cf1bdf9':
            print('MC Score', individual_log['score']['micro_corridor'])
            print('FM', individual_log['first_mile_distance'])
            print(individual_log['micro_corridor_enabled'])

MC Score 0
FM 407.9
True
MC Score 0
FM 1117.0
True
MC Score 0
FM 1112.4
True
MC Score 0
FM 498.9
True
MC Score 0
FM 347.0
True
MC Score 0
FM 298.7
True
MC Score 0
FM 3443.0
True
MC Score 0
FM 396.3
True
MC Score 0
FM 1777.6
True
MC Score 0
FM 2344.0
True
MC Score 0
FM 1795.7
True
MC Score 0
FM 3429.6
True
MC Score 0
FM 345.2
True
MC Score 0
FM 328.9
True
MC Score 0
FM 1136.3
True
MC Score 0
FM 3388.0
True
MC Score 0
FM 3737.8
True
MC Score 0
FM 47.4
True
MC Score 0
FM 735.0
True
MC Score 0
FM 3519.1
True
MC Score 0
FM 2793.8
True
MC Score 0
FM 2793.0
True
MC Score 0
FM 1161.0
True
MC Score 0
FM 1704.4
True
MC Score 0
FM 3443.1
True
MC Score 0
FM 1796.4
True
MC Score 0
FM 3395.4
True
MC Score 0
FM 3737.3
True
MC Score 0
FM 3750.8
True
MC Score 0
FM 3441.7
True
MC Score 0
FM 1152.3
True
MC Score 0
FM 1819.5
True
MC Score 0
FM 3395.4
True
MC Score 0
FM 730.8
True
MC Score 0
FM 88.2
True
MC Score 0
FM 2366.0
True
MC Score 0
FM 1133.5
True
MC Score 0
FM 2793.9
True
MC Score 0
FM 3381.0
True

In [106]:
captains_logs[1][0]

{'captain_id': '62863d9e8f9b4309675e2665',
 'captain_segment': 0,
 'drop_location_score': 0,
 'dropped_to_total_pings_ratio': 0.0,
 'earnings_per_hour': 0,
 'epkm': 14.549728,
 'eta': 0,
 'first_mile_distance': 782.2,
 'handholding': False,
 'high_last_mile_distance_affinity': 0,
 'is_eligible_for_deprioritization': False,
 'is_preferred_route': False,
 'location': {'lat': 28.448918116883434, 'lng': 77.05793883651495},
 'micro_corridor_enabled': False,
 'probability_of_accept': 0,
 'route_preference': {'enabled': False, 'location': {}},
 'score': {'captain_segment': 0.0,
  'deprioritization': 0,
  'drop_location_preference': 0,
  'dropped_to_total_pings_ratio': 0.0,
  'earnings_per_hour': 0,
  'eta': 0,
  'first_mile_distance': 0.0,
  'handholding': 0,
  'high_last_mile_distance_affinity': 0,
  'micro_corridor': 0,
  'probability_of_accept': 0,
  'route_preference': 0,
  'service_affinity': 0},
 'service_affinity': 0,
 'total_score': 0.0}